### Demonstrating some prompting principles and tactics to write effective prompts for large language models.

In [1]:
# !pip install openai --user

In [2]:
import openai
import os

f = open("OPENAI_API_KEY.txt", "r")
openai_api_key = f.read()

#### Throughout I will use OpenAI's gpt-3.5-turbo model and chat completions endpoint https://platform.openai.com/docs/guides/text-generation/chat-completions-api

In [3]:
client = openai.OpenAI(api_key=openai_api_key)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [4]:
# Use delimiters to clearly indicate distinct parts of the input

text = f"""
Generative Pre-trained Transformer 3 (GPT-3) is a large language model \
released by OpenAI in 2020. Like its predecessor, GPT-2, \
it is a decoder-only transformer model of deep neural network, \
which supersedes recurrence and convolution-based architectures \
with a technique known as "attention
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

GPT-3 is a large language model released by OpenAI in 2020, which is a decoder-only transformer model that utilizes attention instead of recurrence and convolution-based architectures.


In [5]:
# "Few-shot" prompting

prompt = f"""
Your task is to answer in a consistent style.

<student>: Teach me about happiness.

<teacher>:  Happiness is like a garden that must be tended \
with care and patience. It is found in the simple joys of life, \
in moments of gratitude and contentment. Just as a flower blooms \
after being nurtured, happiness blossoms when we cultivate positivity \

and kindness in our hearts.

<student>: Teach me about honesty.
"""
response = get_completion(prompt)
print(response)


<teacher>: Honesty is like a sturdy foundation upon which trust and integrity are built. It involves being truthful in our words and actions, even when it may be difficult. Just as a house stands strong with a solid foundation, relationships thrive when honesty is at the core.


In [7]:
# Specify the steps required to complete a task

text = f"""
Humpty Dumpty sat on a wall \
Humpty Dumpty had a great fall; \
All the king's horses and all the king's men \
Couldn't put Humpty together again.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into Spanish.
3 - List each name in the Hindi summary.
4 - Output a json object that contains the following \
keys: Spanish_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Humpty Dumpty had a great fall from a wall and couldn't be put back together by the king's horses and men.

2 - Humpty Dumpty tuvo una gran caída desde una pared y no pudo ser vuelto a armar por los caballos y hombres del rey.

3 - Humpty Dumpty

4 - 
{
  "Spanish_summary": "Humpty Dumpty tuvo una gran caída desde una pared y no pudo ser vuelto a armar por los caballos y hombres del rey.",
  "num_names": 1
}


In [9]:
# Instruct the model to work out its own solution before rushing to a conclusion

prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a wall and I need \
 help working out the total days required to build the wall. 
- 1 man builds the wall in x days \
- 1 woman can build the wall in y days \
What is the total number of days in which 2 man and 3 woman working together can build the wall?

Student's Solution:
1. One man will build in 1 day : 1/x wall
2. One woman in one day can build 1/y wall
3. 2 man and 3 woman working together can build ((2/x) + (3/y)) wall in one day
2 man and 3 woman working togther will complete the wall in: (x*y*2)/((2*y)+(3*x))
"""
response = get_completion(prompt)
print(response)

The student's solution is incorrect. 

The correct way to calculate the total number of days in which 2 men and 3 women working together can build the wall is as follows:

Let's assume that the total work required to build the wall is 1 unit.

1 man can build the wall in x days, so in one day, 1 man can complete 1/x of the work.
Similarly, 1 woman can build the wall in y days, so in one day, 1 woman can complete 1/y of the work.

Therefore, in one day, 2 men and 3 women working together can complete:
2/x (work done by 2 men) + 3/y (work done by 3 women) = (2/x) + (3/y)

To find the total number of days required for 2 men and 3 women to complete the work, we need to divide the total work (1 unit) by the work done in one day by 2 men and 3 women:
Total days = 1 / ((2/x) + (3/y))
Total days = xy / (2y + 3x)

Therefore, the correct formula for the total number of days required for 2 men and 3 women to build the wall is xy / (2y + 3x).


In [12]:
# the previous solution may not be correct therefore fix this by instructing the model to work out its own solution first.

# Instruct the model to work out its own solution before rushing to a conclusion

prompt = f"""

Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```

I'm building a wall and I need \
 help working out the total days required to build the wall. 
- 1 man builds the wall in x days \
- 1 woman can build the wall in y days \
What is the total number of days in which 2 man and 3 woman working together can build the wall?

```

Student's Solution:
```
1. One man will build in 1 day : 1/x wall
2. One woman in one day can build 1/y wall
3. 2 man and 3 woman working together can build ((2/x) + (3/y)) wall in one day
2 man and 3 woman working togther will complete the wall in: (x*y*2)/((2*y)+(3*x))
```

Actual solution:
"""
response = get_completion(prompt)
print(response)

Let's assume the total wall to be built is represented by W.

1. One man builds the wall in x days, so in one day, the man can build 1/x of the wall.
2. One woman builds the wall in y days, so in one day, the woman can build 1/y of the wall.
3. Therefore, 2 men working together can build 2/x of the wall in one day, and 3 women working together can build 3/y of the wall in one day.
4. The total work done by 2 men and 3 women working together in one day is (2/x) + (3/y) of the wall.
5. To find the total number of days required for 2 men and 3 women to build the wall, we need to divide the total work (W) by the combined work rate of 2 men and 3 women.
6. So, the total number of days required = W / ((2/x) + (3/y)) = W / ((2y + 3x) / xy) = W * (xy / (2y + 3x))

Therefore, the total number of days required for 2 men and 3 women working together to build the wall is W * (xy / (2y + 3x)).

So, the student's solution is incorrect.

Actual solution:
Total number of days = W * (xy / (2y + 3x))

I

In [17]:
# Model Limitations: Hallucinations

prompt = f"""
Tell me about Jiffy car by Toyota

"""
response = get_completion(prompt)
print(response)

Jiffy car is a concept car developed by Toyota that was unveiled at the 2017 Tokyo Motor Show. The Jiffy car is a compact electric vehicle designed for short-distance urban transportation. It features a minimalist design with a small footprint, making it ideal for navigating crowded city streets.

The Jiffy car is equipped with advanced technology, including autonomous driving capabilities and a digital interface that allows users to easily book and pay for rides. The interior of the car is designed to be comfortable and spacious, with seating for up to two passengers.

Overall, the Jiffy car by Toyota represents the company's vision for the future of urban mobility, with a focus on sustainability, convenience, and innovation.


In [20]:
# Customize the automated reply to a customer enquiry email reagrding Elevate products and services

In [27]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "soliciting"

# email
email = f"""
Hi  \
Hope you are doing well \
Regarding XYZ's offerings Artificial Intelligence (AI) and Data Science, \
I am looking to explore the same for capabiities for automating contract review. \ 
Can we connect to better exlore the synergies\
Warm regards
Clara
"""

In [28]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a prospective customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their email.
Make sure to use specific details from the email.
Write in a concise and professional tone.
Sign the email as `John with designation as Customer Success Manager`.
Customer review: ```{email}```
Reply sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Subject: Re: Exploring AI and Data Science for Contract Review Automation

Hi Clara,

Thank you for reaching out and expressing interest in exploring XYZ's offerings in Artificial Intelligence (AI) and Data Science for automating contract review. I appreciate your interest in discovering synergies in this area.

I would be more than happy to connect with you to discuss how our solutions can meet your specific needs. When would be a convenient time for you to schedule a call?

Looking forward to exploring the possibilities with you.

Warm regards,
John
Customer Success Manager
